In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import time
from tqdm import tqdm
from IPython.display import Image
import regex as re
from IPython.core.display import display,HTML
from joblib import Parallel, delayed
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
PATH=ChromeDriverManager().install()

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.46M/6.46M [00:02<00:00, 3.06MB/s]


In [2]:
# EL REFUGIO

In [3]:
url = 'https://elrefugio.org/adopta/perros'
driver=webdriver.Chrome(PATH) 
driver.get(url)
time.sleep(3)

muchos=driver.find_elements(By.CLASS_NAME, 'item')
uno = [e.find_element(By.TAG_NAME, 'a') for e in muchos]
links_elrefugio = [e.get_attribute('href') for e in uno]

imagen=[]
nombres= []
carac= []
protec_id=[]
for e in tqdm(links_elrefugio):
    protec_id.append(1)
    driver.get(e)
    box = driver.find_element(By.CLASS_NAME, 'boxSlideFotos')
    img = box.find_element(By.TAG_NAME, 'img')
    imagen.append(img.get_attribute('src'))
    desc = driver.find_element(By.CLASS_NAME, 'txtDoble')
    nombres.append(desc.find_element(By.TAG_NAME, 'h1').text)
    todo = desc.find_elements(By.TAG_NAME, 'li')
    for i in todo:
        i = i.text
        i = i.split(':')
        i.pop(0)
        carac += i
        
driver.quit()

n = 6
caracter = [carac[i:i + n] for i in range(0, len(carac), n)]

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [00:38<00:00,  1.86it/s]


In [6]:
colnames= ['SEXO', 'EDAD', 'NACIMIENTO', 'RAZA', 'TAMAÑO', 'CARACTER']
perros_1 = pd.DataFrame(caracter, columns=colnames)

perros_1['NOMBRE']= nombres
perros_1['PROTECTORA']= protec_id
perros_1['LINK']= links_elrefugio

sexo= []
for e in perros_1.SEXO:
    e = e.lower()
    e= e.strip()
    sexo.append(e)
perros_1['SEXO']= sexo

raza= []
for e in perros_1.RAZA:
    e = e.lower()
    e= e.strip()
    raza.append(e)
perros_1['RAZA']= raza

tam= []
for e in perros_1.TAMAÑO:
    e = e.lower()
    e= e.strip()
    tam.append(e)
perros_1['TAMAÑO']= tam

car= []
for e in perros_1.CARACTER:
    e = e.lower()
    e= e.strip()
    car.append(e)
perros_1['CARACTER']= car

nom= []
for e in perros_1.NOMBRE:
    e = e.capitalize()
    e= e.strip()
    nom.append(e)
perros_1['NOMBRE']= nom

nac= []
for e in perros_1.NACIMIENTO:
    e= e.strip()
    nac.append(e)
perros_1['NACIMIENTO']= nac




images1 = imagen
perros_1['FOTO'] = images1

# convert your links to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

pd.set_option('display.max_colwidth', None)

image_cols = ['FOTO']  #<- define which columns will be used to convert to html

# Create the dictionariy to be passed as formatters
format_dict = {}
for e in image_cols:
    format_dict[e] = path_to_image_html


#display(HTML(perros.to_html(escape=False ,formatters=format_dict)))

perros_1.RAZA.replace({'galga': 'galgo', 'mestiza': 'mestizo'}, inplace= True)
perros_1.TAMAÑO.replace({'medio': 'mediano', 'mediana': 'mediano'}, inplace= True)
perros_1.CARACTER.replace({'muy cariñosa': 'cariñosa', 'timida': 'tímida'}, inplace= True)
perros_1['NACIMIENTO'] = pd.to_datetime(perros_1['NACIMIENTO'])
perros_1['NACIMIENTO'] = (perros_1['NACIMIENTO']).dt.year
perros_1['HOY'] = pd.datetime.now().date()
perros_1['HOY'] = pd.to_datetime(perros_1['HOY'])
perros_1['HOY'] = (perros_1['HOY']).dt.year
perros_1['AÑOS'] = (perros_1['HOY'] - perros_1['NACIMIENTO'])
perros_1.drop(['NACIMIENTO', 'HOY', 'EDAD'], axis = 1, inplace = True)


In [8]:
perros_1.to_csv('perros_1.csv', index= False)

In [27]:
url = 'https://elrefugio.org/adopta/gatos'
driver=webdriver.Chrome(PATH) 
driver.get(url)
time.sleep(3)

mas = driver.find_element(By.XPATH, '//*[@id="divVerMas"]/a')
mas.click()
time.sleep(3)
muchos=driver.find_elements(By.CLASS_NAME, 'item')
uno = [e.find_element(By.TAG_NAME, 'a') for e in muchos]
links_elrefugio = [e.get_attribute('href') for e in uno]
links_elrefugio.remove(None)

imagen=[]
nombres= []
carac= []
protec_id=[]
for e in tqdm(links_elrefugio):
    protec_id.append(1)
    driver.get(e)
    time.sleep(1)
    box = driver.find_element(By.CLASS_NAME, 'boxSlideFotos')
    img = box.find_element(By.TAG_NAME, 'img')
    imagen.append(img.get_attribute('src'))
    desc = driver.find_element(By.CLASS_NAME, 'txtDoble')
    nombres.append(desc.find_element(By.TAG_NAME, 'h1').text)
    todo = desc.find_elements(By.TAG_NAME, 'li')
    for i in todo:
        i = i.text
        i = i.split(':')
        i.pop(0)
        carac += i
        
driver.quit()

n = 5
caracter = [carac[i:i + n] for i in range(0, len(carac), n)]

100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [00:51<00:00,  1.52s/it]


In [50]:
colnames= ['SEXO', 'EDAD', 'NACIMIENTO', 'RAZA', 'CARACTER']
gatos_1 = pd.DataFrame(caracter, columns=colnames)

gatos_1['NOMBRE']= nombres
gatos_1['PROTECTORA']= protec_id
gatos_1['LINK']= links_elrefugio

sexo= []
for e in gatos_1.SEXO:
    e = e.lower()
    e= e.strip()
    sexo.append(e)
gatos_1['SEXO']= sexo

raza= []
for e in gatos_1.RAZA:
    e = e.lower()
    e= e.strip()
    raza.append(e)
gatos_1['RAZA']= raza


car= []
for e in gatos_1.CARACTER:
    e = e.lower()
    e= e.strip()
    car.append(e)
gatos_1['CARACTER']= car

nom= []
for e in gatos_1.NOMBRE:
    e = e.capitalize()
    e= e.strip()
    nom.append(e)
gatos_1['NOMBRE']= nom

nac= []
for e in gatos_1.NACIMIENTO:
    e= e.strip()
    nac.append(e)
gatos_1['NACIMIENTO']= nac


gatos_1.CARACTER.replace({'timida': 'tímida', 'cariñoso y tímida': 'cariñosa y tímida'}, inplace= True)
gatos_1['NACIMIENTO'] = pd.to_datetime(gatos_1['NACIMIENTO'])
gatos_1['NACIMIENTO'] = (gatos_1['NACIMIENTO']).dt.year
gatos_1['HOY'] = pd.datetime.now().date()
gatos_1['HOY'] = pd.to_datetime(gatos_1['HOY'])
gatos_1['HOY'] = (gatos_1['HOY']).dt.year
gatos_1['AÑOS'] = (gatos_1['HOY'] - gatos_1['NACIMIENTO'])
gatos_1.drop(['NACIMIENTO', 'HOY', 'EDAD', 'RAZA'], axis = 1, inplace = True)

images1 = imagen
gatos_1['FOTO'] = images1

# convert your links to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

pd.set_option('display.max_colwidth', None)

image_cols = ['FOTO']  #<- define which columns will be used to convert to html

# Create the dictionariy to be passed as formatters
format_dict = {}
for e in image_cols:
    format_dict[e] = path_to_image_html


#display(HTML(perros.to_html(escape=False ,formatters=format_dict)))


In [52]:
gatos_1.to_csv('gatos_1.csv', index= False)

In [53]:
gatos_1.head()

,SEXO,RAZA,CARACTER,NOMBRE,PROTECTORA,LINK,AÑOS,FOTO
0,hembra,común europeo,tranquila,Maskra,1,https://elrefugio.org/ficha/maskra,1,https://elrefugio.org/contenido/animales/thumbsFicha/El-Refugio-Maskraweb.jpg
1,macho,común europeo,cariñoso,Peach,1,https://elrefugio.org/ficha/peach,6,https://elrefugio.org/contenido/animales/thumbsFicha/El-Refugio-Peachweb.jpg
2,hembra,común europeo,cariñosa,Kinya,1,https://elrefugio.org/ficha/kinya,1,https://elrefugio.org/contenido/animales/thumbsFicha/El-Refugio-Kinyaweb_1.jpg
3,hembra,común europeo,asustadiza,Bentú,1,https://elrefugio.org/ficha/bentu,7,https://elrefugio.org/contenido/animales/thumbsFicha/El-Refugio-Bentuweb.jpg
4,hembra,común europeo,independiente,Sachet,1,https://elrefugio.org/ficha/sachet,8,https://elrefugio.org/contenido/animales/thumbsFicha/El-Refugio-Sachetweb.jpg
